# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [ ]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [2]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving yellow_tripdata_2022-01.parquet to yellow_tripdata_2022-01.parquet


In [3]:
# Load parquet file into a Pandas DataFrame
file_name = "yellow_tripdata_2022-01.parquet"

df = pd.read_parquet(file_name)

In [4]:
# Display the first few rows of the dataset
print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            68           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [5]:
# Drop rows with missing values.
df2 = df.dropna()

In [6]:
# Create new feature, 'trip_duration'.
# First we check the format of the pickup and dropoff datetime columns
print(df2[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].dtypes)

# Create 'trip_duration' in seconds
df2.loc[:, 'trip_duration'] = (df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']).dt.total_seconds()

# Display
print(df2.head())


tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
dtype: object
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            6

<ipython-input-6-dea461e8b463>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, 'trip_duration'] = (df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']).dt.total_seconds()


In [7]:
# Create a list called feature_col to store column names
feature_col = df2.columns.tolist()

Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [8]:
# Split dataset into training and test sets
X = df2.drop(columns=['trip_duration'])
y = df2['trip_duration']

# Split dataset: Training (80%) and testing (20%)
# Fixed random state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

In [10]:
# Create a baseline for mean absolute error of total amount

# Calculate the mean total fare from the training dataset
mtfare = np.mean(y_train)

# Apply the above mean to all test samples
# This creates an array which is the same size as y_test and fills every element with mtfare
# Which will ensure that we can compare the baseline prediction mtfare in this case with actual
# values using the mean absolute error.
y_mtfare = np.full_like(y_test, mtfare)

# Evaluate the baseline model using MAE - Mean Absolute Error
bmae = mean_absolute_error(y_test, y_mtfare)

print(bmae)

# Interpretation:
# After running the above code we get a value of 529.398 which means that the mean total fare
# predicted is off by ~$530 on average from the actual fare of each trip.
# This suggests that the model is not very accurate, a better model will take into account
# other factors before making a prediction. Error is high and we need to build a model that
# takes into account other factors such as distance, location instead of just the duration of the trip.
# The prediction is based only on average fare from the training dataset, it predicts the same
# value for every trip.


529.3980471931345


With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


In [11]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.

# Check which columns are continuous and categorical.
df_info = df2.info()

# Print column names
print(df2.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 2392428 entries, 0 to 2392427
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [12]:
# Continuous and Categorical Variables
conti = ['fare_amount', 'total_amount', 'trip_distance', 'trip_duration', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
cat = ['VendorID', 'payment_type', 'passenger_count','store_and_fwd_flag']

print(conti)
print(cat)

X = df2[conti + cat]
y = df2['trip_duration']

# Split dataset: Training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

['fare_amount', 'total_amount', 'trip_distance', 'trip_duration', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
['VendorID', 'payment_type', 'passenger_count', 'store_and_fwd_flag']


In [14]:
# Column Transformer
# StandardScaler to continuous features
# OneHotEncoder to categorical features
# Transform the data by standardizing them, making them comparable in scale
# Convert categorical features into numerical format so the machine learning model
# can understand and process them.
preproc = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), conti),
        ('cat', OneHotEncoder(), cat)
    ])

# Apply ColumnTransformer on X
X_train_processed = preproc.fit_transform(X_train)

In [15]:
# Create a pipeline object containing the column transformations and regression
# model.

# Apply the column transformer (preprocessing)
# Apply Linear Regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preproc),
    ('regressor', LinearRegression())
])

In [16]:
# Fit the pipeline on the training data.
print(df2.columns)
print(X_train.columns)
pipeline.fit(X_train, y_train)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'trip_duration'],
      dtype='object')
Index(['fare_amount', 'total_amount', 'trip_distance', 'trip_duration',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'congestion_surcharge', 'airport_fee',
       'VendorID', 'payment_type', 'passenger_count', 'store_and_fwd_flag'],
      dtype='object')


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['fare_amount',
                                                   'total_amount',
                                                   'trip_distance',
                                                   'trip_duration', 'extra',
                                                   'mta_tax', 'tip_amount',
                                                   'tolls_amount',
                                                   'improvement_surcharge',
                                                   'congestion_surcharge',
                                                   'airport_fee']),
                                                 ('cat', OneHotEncoder(),
                                                  ['VendorID', 'payment_type',
                                                   'passenger_count',
                                                   'store_and_fwd_flag'])])),
                ('regressor', LinearRegression())])

In [17]:
# Make predictions on the test data.
y_pred = pipeline.predict(X_test)

# Display the predictions
print(y_pred)

# Calculate Mean Absolute Error (MAE)
model_mae = mean_absolute_error(y_test, y_pred)

print(model_mae)

# Interpretations:
# The value of model_mae was around 1.723e-12, this means the model is making way accurate
# predictions that the baseline.
# the average error is very small essentially zero, which means that the predicted trip durations
# are very identical to the actual trip duration. This shows that the model is very accurate.

[592. 471. 560. ... 712. 647. 372.]
1.7239482279272938e-12


Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [18]:
# Create a pipeline with preprocessing and Random Forest Regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preproc),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [19]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['fare_amount',
                                                   'total_amount',
                                                   'trip_distance',
                                                   'trip_duration', 'extra',
                                                   'mta_tax', 'tip_amount',
                                                   'tolls_amount',
                                                   'improvement_surcharge',
                                                   'congestion_surcharge',
                                                   'airport_fee']),
                                                 ('cat', OneHotEncoder(),
                                                  ['VendorID', 'payment_type',
                                                   'passenger_count',
                                                   'store_and_fwd_flag'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [22]:
# Make predictions on the test data
y_pred_rf = pipeline.predict(X_test)

# Evaluate the model using mean absolute error
rf_mae = mean_absolute_error(y_test, y_pred_rf)

print("Random Forest Regressor MAE:", rf_mae)

# Interpretation:
# The model predicts trip duration very accurately
# with an average mistake of just 0.15 seconds.
# This means the predictions are very close to the actual trip durations
# which means that the model is actually doing a very good job.

Random Forest Regressor MAE: 0.14711287268593043


Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [23]:
# Define the hyperparameters to tune.
# Try different combinations of n_estimators, max_depth and min_sample_split
# to find the best settings for the RandomForestRegressor Model
# Parameter grid
param_grid = {
    # Number of trees in the forest
    'n_estimators': [50, 100, 200],
    # Maximum depth of each tree
    'max_depth': [10, 20, 30],
    # Minimum number of samples required to split an internal node
    'min_samples_split': [2, 5, 10]
}

# Create the Random Forest model
rf_model = RandomForestRegressor(random_state=42)

In [ ]:
# Perform grid search to find the best hyperparameters. This could take a while.
# Perform Grid Search with cross-validation
# Test each combination using GridSearchCV and pick the one which gives the best
# performance, that is, lowest mae, mean absolute error
# cv=3, split the training data into three parts for cross-validation,
# to validate the model multiple times
# to make sure the best parameters work well across different subsets of the
# data, reducing the risk of overfitting to a specific portion of the dataset.
# The model is trained 3 times each time using 2 parts for training and 1 part for validation.
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=3, scoring='neg_mean_absolute_error',
                           n_jobs=-1, verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_processed, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


In [ ]:
# Get the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Get the best model
# use this to make predictions on new data
best_rf_model = grid_search.best_estimator_

In [ ]:
# Fit the best classifier on the training data.
# Find the best settings and store it as best_params
best_params = grid_search.best_params_

# With these best parameters which we got above, train a new RandomForestRegressor.
best_rf_model = RandomForestRegressor(**best_params, random_state=42)

# Fit the model
best_rf_model.fit(X_train_processed, y_train)

In [ ]:
# Make predictions on the test data
# Transform the test data using the same preprocessor
X_test_processed = preproc.transform(X_test)

# Make predictions on the test data
y_pred = best_rf_model.predict(X_test_processed)

In [ ]:
# Calculate the mean absolute error (MAE)
model_mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE) of the optimized Random Forest model: {model_mae:.4f}")

In [ ]:
# Interpretation:
# We tested different settings for a random forest model to find
# which ones are the best ones for making accurate predictions.
# At first we created a list of options for how many trees to use and how deep they should be.
# Then we used GridSearchCV to try out all these options and find the best combination.
# Then we built a new model, best_rf_model using these best settings
# and trained it on all the training data.
# Before making predictions we processed the test data the same way
# as the training data preproc.transform and then used the model to
# predict trip durations best_rf_model.predict.
# Finally, we measured how accurate the predictions were by checking the
# Mean Absolute Error which tells us how far off our predictions were from the actual trip times.

# This helped us in getting the best settings so that the model could make more accurate predictions.
# We are able to test different options and make sure that the model works well on all data.

# The downside of this process is that it is time consuming, took us more than 20 mins to run one line of code.
# And this is different from the GridSearch fit code which took more than 50 mins to run.
# may need more powerful computing resources.
# With everything it helped us create a model that predicts trip duration as accurately as possible.